# Задание 4 

Для анализа необходимо импортировать библиотеку pandas.

In [211]:
import pandas as pd

Для загрузки используется метод Pandas read.csv().

In [212]:
test_df=pd.read_csv('test.csv')

Метод .head(5) используется для вывода первых пяти строк фрейма данных. 
Этот метод даёт представление о данных(столбцы и тип возращаемых данных). 

In [213]:
test_df.head(5)

,install_date,country,campaign_id,campaign_name,installs
0,2022-10-15,CH,6488890999,I3cE61G6kA1ihaBcdDAA,256
1,2022-08-30,CX,6896175429,eH0Bk-G..d7GCgA5kff2,1022
2,2022-05-23,BY,2922650240,DF6_h_E.bf0_._cA2C7b,3837
3,2023-01-23,CX,8282076813,GBFgbGb4ee2.G.G2aAFK,272
4,2022-11-27,BR,7617371497,H2kb10__.fGHc88f77fb,3984


Посмотреть на размер данных (количество строк, колонок).

In [214]:
test_df.shape

(100000, 5)

Для просмотра статистической сводки каждого столбца, чтобы узнать распределение данных в каждом столбце используется метод describe( ). 
Для категориальных признаков(строки и столбцы не содержащие чисел) этот метод показывает: - Сколько уникальных значений в - - наборе данных - unique; 
top значения;
частота появления значений - freg.

In [215]:
test_df.describe(include = "all")

,install_date,country,campaign_id,campaign_name,installs
count,100000,99005,99998,100000,1.000000e+05
unique,368,16,99995,100000,NaN
top,2022-08-28,BY,9935450190,I3cE61G6kA1ihaBcdDAA,NaN
freq,325,8126,2,1,NaN
mean,NaN,NaN,NaN,NaN,4.542868e+04
std,NaN,NaN,NaN,NaN,9.603780e+06
min,NaN,NaN,NaN,NaN,-1.000000e+00
25%,NaN,NaN,NaN,NaN,1.212000e+03
50%,NaN,NaN,NaN,NaN,2.484000e+03
75%,NaN,NaN,NaN,NaN,3.744000e+03


Уже по этому методу можно наблюдать, что минимальное значение столбца installs является отрицательным числом. Запомним это и вернёмся к этому позже.

Метод info( ) - показывает информацию о наборе данных, индекс, столбцы и тип данных, ненулевые значения и использование памяти.

In [216]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   install_date   100000 non-null  object
 1   country        99005 non-null   object
 2   campaign_id    99998 non-null   object
 3   campaign_name  100000 non-null  object
 4   installs       100000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.3+ MB


По этому методу заметно, что есть пропущенные значения в колонках country,campaign_id.

### Отсутствующие значения

Найдём количество пропущенных значений в колонках.

In [217]:
test_df.isna().sum()

install_date       0
country          995
campaign_id        2
campaign_name      0
installs           0
dtype: int64

Для дальнейшего анализа заменим пропущенные значения в колонках на 0.

In [218]:
test_df[['campaign_id']]=test_df[['campaign_id']].fillna(0)
test_df[['country']]=test_df[['country']].fillna(0)

Проверим,что метод сработал.

In [219]:
test_df.isna().sum()

install_date     0
country          0
campaign_id      0
campaign_name    0
installs         0
dtype: int64

Создадим бинарный флаг campaign_id_nan_flag, чтобы обозначить строки, у которых не проставлен campaign_id.

In [220]:
test_df['campaign_id_nan_flag'] = (test_df['campaign_id'] == 0).astype("int")
test_df['campaign_id_nan_flag'].value_counts()

0    99998
1        2
Name: campaign_id_nan_flag, dtype: int64

Тоже самое сделаем для колонки country.

In [222]:
test_df['country_nan_flag'] = (test_df['country'] == 0).astype("int")
test_df['country_nan_flag'].value_counts()

0    99005
1      995
Name: country_nan_flag, dtype: int64

### Дубликаты

Проверяем есть ли у нас дублирующиеся строки.

In [223]:
duplicateRows = test_df[test_df.duplicated()]

print(duplicateRows)

Empty DataFrame
Columns: [install_date, country, campaign_id, campaign_name, installs, campaign_id_nan_flag, country_nan_flag]
Index: []


Полного дублирования нет.

Значения в колонке campaign_name должны быть уникальными и соответствовать campaign_id. Проверим.

In [224]:
duplicate_campaign_name=test_df[test_df.duplicated(['campaign_name'])]
print(duplicate_campaign_name)

Empty DataFrame
Columns: [install_date, country, campaign_id, campaign_name, installs, campaign_id_nan_flag, country_nan_flag]
Index: []


В колонке campaign_id должны быть уникальные значения. 

In [225]:
duplicate_campaign_id=test_df[test_df.duplicated(['campaign_id'])]
print(duplicate_campaign_id)

      install_date country campaign_id         campaign_name  installs  \
84      2022-06-06      CA           0  h21iChHb974dDb9F3E.g      3752   
59646   2023-02-05      ID  9177739839  IE1E9HB5gF.-a23aIK63      2982   
61645   2022-04-07      BY  2905851120  63baEbg0adgFhF0489G5       957   
75611   2022-03-24      UK  9935450190  a0kBH6A46KFd5Ib42.4-      4084   

       campaign_id_nan_flag  country_nan_flag  
84                        1                 0  
59646                     0                 0  
61645                     0                 0  
75611                     0                 0  


Видим,что есть повторяющиеся значения campaign_id. Значение 0-нас не интересует в данном случае, потому что это строки, где были отсутствующие значения campaign_id и эта проблема решится отдельно.

Рассмотрим строки с повторениями campaign_id.
Указано в задании,что название рекламной кампании должно соответствует своему campaign_id, а ниже можем наблюдать разные названия для определенного id. Ошибка

In [226]:
test_df[test_df['campaign_id'] == '9177739839']

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
31267,2022-08-19,ID,9177739839,GGfADG597cG84206c.5k,4322,0,0
59646,2023-02-05,ID,9177739839,IE1E9HB5gF.-a23aIK63,2982,0,0


In [227]:
test_df[test_df['campaign_id'] == '2905851120']

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
33478,2023-02-05,CA,2905851120,IHiAa2-KF21CHC-6B3bC,3916,0,0
61645,2022-04-07,BY,2905851120,63baEbg0adgFhF0489G5,957,0,0


In [228]:
test_df[test_df['campaign_id'] == '9935450190']

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
33477,2022-05-10,UK,9935450190,k0fH_1EeCf877772c9aF,725,0,0
75611,2022-03-24,UK,9935450190,a0kBH6A46KFd5Ib42.4-,4084,0,0


###	Достоверность данных

Посмотрим типпы данных столбцов.

In [229]:
test_df.dtypes

install_date            object
country                 object
campaign_id             object
campaign_name           object
installs                 int64
campaign_id_nan_flag     int32
country_nan_flag         int32
dtype: object

Для дальнейшего анализа попробуем привести данные в тип данных, кторый подходит по смыслу. Так они будут правильно сортироваться и будет легче найти значения, которые не подходят по типу или по смыслу.

Колонку install_date приводим к типу datetime.

In [154]:
test_df['install_date']=pd.to_datetime(test_df['install_date'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2300-12-10 00:00:00

Видим,что произошла ошибка "Out of bounds nanosecond timestamp: 2300-12-10 00:00:00"". 

Мы получаем ошибку OutOfBoundsDatetime , потому что данная отметка времени больше, чем максимальная отметка времени в наносекундах, разрешенная pandas.

Ну и в целом, как мы можем в дальнейшем анализировать данную дату в маркетинге, если она еще не наступила.

In [230]:
test_df[test_df['install_date'] == '2300-12-10'] 

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
43018,2300-12-10,IN,6310005890,3B4KCeB0AAfCie1i3ceH,1701,0,0
68512,2300-12-10,CH,5829506265,42eccAKk8bcH2A0a2bb0,3985,0,0
98220,2300-12-10,CC,3716295537,95cDB0G8KK8f2HKG3gbe,1301,0,0


Представлено 3 строки с данной датой.

In [231]:
test_df['install_date']=pd.to_datetime(test_df['install_date'],errors = 'coerce')

In [232]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   install_date          99997 non-null   datetime64[ns]
 1   country               100000 non-null  object        
 2   campaign_id           100000 non-null  object        
 3   campaign_name         100000 non-null  object        
 4   installs              100000 non-null  int64         
 5   campaign_id_nan_flag  100000 non-null  int32         
 6   country_nan_flag      100000 non-null  int32         
dtypes: datetime64[ns](1), int32(2), int64(1), object(3)
memory usage: 3.4+ MB


In [233]:
test_df[test_df['install_date'].isna()]

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
43018,NaT,IN,6310005890,3B4KCeB0AAfCie1i3ceH,1701,0,0
68512,NaT,CH,5829506265,42eccAKk8bcH2A0a2bb0,3985,0,0
98220,NaT,CC,3716295537,95cDB0G8KK8f2HKG3gbe,1301,0,0


Отсортируем данные по колонке install_date по возрастанию, а затем -по убыванию. Чтобы посмотреть промежуток времени,для которого нам представлены даты.

In [234]:
test_df.sort_values(by=['install_date'])

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
99555,1970-01-01,IN,5916072091,aI64GD39hB08af.k4c7G,1703,0,0
99627,1970-01-01,BY,4798685209,E7kB1e49.IDKBh9fHkd-,4928,0,0
76114,2022-03-22,US,6684019821,G3I880_--3k-FDd1Digh,2736,0,0
83398,2022-03-22,CX,8074416497,IKg3GKeF-F4hFGCfHEAe,3813,0,0
73277,2022-03-22,CC,3742958882,Ed.Bf8D-64CE.aaG-6B-,407,0,0
...,...,...,...,...,...,...,...
2520,2023-03-22,CU,4050917027,63K3D_5hggDH4Khi1C3G,4598,0,0
14544,2023-03-22,GR,3022869594,-0CIbe.e10bE5f8hkb7D,2312,0,0
43018,NaT,IN,6310005890,3B4KCeB0AAfCie1i3ceH,1701,0,0
68512,NaT,CH,5829506265,42eccAKk8bcH2A0a2bb0,3985,0,0


In [235]:
test_df.sort_values(by=['install_date'],ascending=False)

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
71649,2023-03-22,CU,4086419030,0a1_gf253-gfK9IagE4F,4748,0,0
87659,2023-03-22,CX,1881983832,_0D3ah.fCkFEFC.E0Aeb,1766,0,0
46249,2023-03-22,GR,7229294286,bE21BI0bC2D4_951hAf_,4535,0,0
80422,2023-03-22,US,9570967101,FAbcibIGGHg2d75bb3h0,2058,0,0
3818,2023-03-22,CX,3050672431,KGAI8Ie.h7EkB9kEi9gD,2862,0,0
...,...,...,...,...,...,...,...
99627,1970-01-01,BY,4798685209,E7kB1e49.IDKBh9fHkd-,4928,0,0
99555,1970-01-01,IN,5916072091,aI64GD39hB08af.k4c7G,1703,0,0
43018,NaT,IN,6310005890,3B4KCeB0AAfCie1i3ceH,1701,0,0
68512,NaT,CH,5829506265,42eccAKk8bcH2A0a2bb0,3985,0,0


При сортировке по возрастанию можно увидеть две строки, где выбивается дата '1970-01-01' из основного промежутка 2022-2023г. Нам для анализа эти даты не нужны или записались неверные даты.

Так данные находятся в нужном формате "yyyy-mm-dd"

Колонка installs имеет тип данных int64.
installs - количество установок в разрезе install_date, country, campaign_id.
Значит колонка должна содержать положительные значения и по значению не равные 0.
Проверим это.

In [185]:
print(len(test_df[test_df['installs'] >= 0]))

99027


Видим, что количество положительных значений в колонке installs меньше, чем общее количество значений. 
Значит,есть значения, которые не подходят нам по смыслу, посмотрим их.

In [236]:
test_df[test_df['installs'] < 0]

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
144,2022-09-01,US,1255485065,hF.9k.cC0c4ABDF8AEhH,-1,0,0
161,2022-06-19,GR,3234596177,K07CbEi.-796a1Ed53dE,-1,0,0
168,2022-10-12,CU,4460545350,E9Kb.0k-2bai6a_GCgkI,-1,0,0
220,2023-01-09,IN,5251414190,19Aik-HFc0B.K_1-Fc3F,-1,0,0
226,2022-09-23,ID,9959072621,8gGH4hI90i5kbAkCI.F-,-1,0,0
...,...,...,...,...,...,...,...
99518,2022-04-06,CA,8090906063,F1Fc0C358HHbE7c0aAIa,-1,0,0
99608,2023-01-17,IN,6908595158,E06HhEkG1D.HAghI-b9A,-1,0,0
99644,2023-01-21,GR,4425343535,_IK7FhAfC2e9f_7DIie8,-1,0,0
99687,2023-01-30,US,3726300487,e5A4dcdIDh0DCb9fikcC,-1,0,0


Колонка installs содержит значения -1. Неверные значения.

In [237]:
test_df[test_df['installs'] == 0]

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
733,2022-07-04,GR,7246022772,3871A5.cfeFC8kHFb7AF,0,0,0
5382,2022-09-04,BR,7840706469,ah3E_5c6F3fF-e86fbdg,0,0,0
6961,2022-05-09,CC,9434409190,d5gEcG56.K7bBf2I2498,0,0,0
11368,2022-05-28,CH,1867270296,-BkaHC5BgBFE.1.0.AGA,0,0,0
36217,2023-02-10,59,9784301183,hi50-BHiFkBh.-FHhcik,0,0,0
43167,2023-02-13,CX,5106711273,-eCb3bAHI11ebic6A051,0,0,0
44526,2022-09-18,US,1500228011,BaK777b78b_-fDdHB4ce,0,0,0
49039,2022-07-01,UK,3626697462,i4kE4keFB0bB75Df1IKc,0,0,0
70615,2023-02-08,EU,9656810820,facAkK11CGc8716712D7,0,0,0
74297,2022-10-28,GR,6852471098,963G.KhA31068I-BKg08,0,0,0


Так же есть данные со значением 0 в этой колонке. Мне кажется, что они не несут смысла.

Отсортируем разным порядком данные по колонке installs.

In [238]:
test_df[test_df['installs'] > 0].sort_values(by=['installs'])

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
18641,2022-04-22,ID,5934108285,GA05_45d4e9944C_bG2E,1,0,0
41079,2022-07-01,CC,6517521356,K9E.g6DFiDF3hB_8gi7B,1,0,0
67292,2022-10-19,ID,6513499394,DC_IaeF_5B9biHAEH6CA,1,0,0
768,2023-01-09,CC,4537087271,1DeKD5ck8a.B18kE78EK,1,0,0
86919,2022-04-27,GR,2915166855,6dBK_fGf_597abAagf99,1,0,0
...,...,...,...,...,...,...,...
70400,2022-11-17,BY,4177671871,4CC-H5E-_eDhcBK31f6a,5000,0,0
30123,2022-06-22,CC,1822561467,56hE-h74CFI9aG3kID2k,5000,0,0
48588,2022-07-27,ID,4835432137,f6kB-B4hFKHIdC2Cd7Aa,5000,0,0
47010,2022-12-18,BY,9419499089,A6A8_8CGdCKd733.-9dC,2147483647,0,0


In [239]:
test_df[test_df['installs'] > 0].sort_values(by=['installs'],ascending=False)

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
14,2023-03-15,US,1371207243,B-8HhcHKB9bd.94c46-1,2147483647,0,0
47010,2022-12-18,BY,9419499089,A6A8_8CGdCKd733.-9dC,2147483647,0,0
53757,2022-05-17,BR,6764734003,05fAeg56CdF8.hd.-a5A,5000,0,0
31244,2023-01-26,CC,9089631021,cdDD3faCcH7GfIGa2_E2,5000,0,0
2041,2023-01-18,BR,7125225835,D.7AB681GeFC06H_f7G7,5000,0,0
...,...,...,...,...,...,...,...
18641,2022-04-22,ID,5934108285,GA05_45d4e9944C_bG2E,1,0,0
69007,2022-09-28,CH,3225538365,h-Afihk9caE160cc7C9d,1,0,0
34544,2022-06-29,IN,6209894138,FI1211K2HG_6C-F74K-g,1,0,0
13481,2023-03-21,US,5160176082,f07C8iB7h5gbA1CHdKKb,1,0,0


Наблюдаем при сортировке по убыванию 2 значения в колонке installs 2147483647. Слишком большие числа, которые выбиваются из остальных.

Проанализируем колонку campaign_name.

In [240]:
test_df.sort_values(by=['campaign_name'])

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
36150,2022-12-09,UK,6041707964,$CAMPAIGN_NAME$,1892,0,0
44797,2022-04-17,CX,4447229522,---24gh.HHbBbbh5a2iB,3876,0,0
72091,2022-09-23,CU,2172099851,---CH8i-D6gCB.bHF-K5,4669,0,0
65624,2022-09-22,GR,2547981534,---FA8f734_6iE27__bi,3305,0,0
80850,2023-01-27,CU,4140981396,--.44hA2gB0Fb6IbkKgf,2160,0,0
...,...,...,...,...,...,...,...
91118,2022-10-14,BR,5534702023,kkh9E-fH8bDE7-Ie868h,2969,0,0
98091,2022-09-17,ID,5214046564,kkhcBk3cC4GEHDc3K0fK,3973,0,0
18442,2023-01-06,CX,2370746794,kkhi._5D94_g6k19H84f,4444,0,0
16545,2022-11-23,ID,2707566603,kkifKBHbc0A0_5iE-KFa,319,0,0


In [251]:
test_df.sort_values(by=['campaign_name'],ascending=False)

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
70501,2023-02-03,BR,3268642679,kkkaf6G7.BfFI50aedAI,169,0,0
16545,2022-11-23,ID,2707566603,kkifKBHbc0A0_5iE-KFa,319,0,0
18442,2023-01-06,CX,2370746794,kkhi._5D94_g6k19H84f,4444,0,0
98091,2022-09-17,ID,5214046564,kkhcBk3cC4GEHDc3K0fK,3973,0,0
91118,2022-10-14,BR,5534702023,kkh9E-fH8bDE7-Ie868h,2969,0,0
...,...,...,...,...,...,...,...
80850,2023-01-27,CU,4140981396,--.44hA2gB0Fb6IbkKgf,2160,0,0
65624,2022-09-22,GR,2547981534,---FA8f734_6iE27__bi,3305,0,0
72091,2022-09-23,CU,2172099851,---CH8i-D6gCB.bHF-K5,4669,0,0
44797,2022-04-17,CX,4447229522,---24gh.HHbBbbh5a2iB,3876,0,0


Можно заметить значение '𝐶𝐴𝑀𝑃𝐴𝐼𝐺𝑁𝑁𝐴𝑀𝐸'. И эта колонка содержит значения с разной длиной, символами. Уточнить бы какие символы недопустимы для значений и может написать регулярное выражение по которому отбирать верные значения для этой колонки.

campaign_id - числовой идентификатор рекламной кампании. Попробуем привести данную колонку к типу int. Так как обычно id представляют в данном типе. Но числа в данной колонке очень большые проэтому приведем к типу int64.

In [248]:
test_df['campaign_id']=test_df['campaign_id'].astype('str').astype('int64')

ValueError: invalid literal for int() with base 10: '(cid)'

In [253]:
test_df['campaign_id']=test_df['campaign_id'].astype('str')

Наблюдаем ошибку. Не получается значение '(cid)' привести к int. Само это значение-это строка,что не подходит по смыслу.
Проверим, может есть еще в жтой колонке строковые значения.

In [254]:
test_df.sort_values(by=['campaign_id'])

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
45236,2022-05-16,ZZ,$CAMPAIGN_ID$,id.FH0F23ci8bG2hD693,4985,0,0
16636,2022-11-18,GR,(cid),B3a798eD1C0D-EkDH5E4,1392,0,0
84,2022-06-06,CA,0,h21iChHb974dDb9F3E.g,3752,1,0
9,2022-04-17,CC,0,.3k8Egg2ahGH1DF08Dck,2264,1,0
45538,2022-08-20,US,1000048325,4AibH91K4Ifk81C7_c.1,107,0,0
...,...,...,...,...,...,...,...
23000,2022-07-11,BY,9999538518,7.05465kE0KaI2BaGckd,4945,0,0
2702,2023-03-10,CA,9999553434,6aEk1E2k0bf8d3A4f4If,1485,0,0
82150,2022-12-15,CH,9999592791,DKCG6GfE25b1a93A70AB,2542,0,0
81712,2022-06-21,IN,9999608363,BG5DcbEBe5BhcCb81eB2,4336,0,0


In [255]:
test_df.sort_values(by=['campaign_id'],ascending=False)

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
12956,2022-11-12,ID,9999921998,kEb5fB94Hb1g.c84GDb_,3246,0,0
81712,2022-06-21,IN,9999608363,BG5DcbEBe5BhcCb81eB2,4336,0,0
82150,2022-12-15,CH,9999592791,DKCG6GfE25b1a93A70AB,2542,0,0
2702,2023-03-10,CA,9999553434,6aEk1E2k0bf8d3A4f4If,1485,0,0
23000,2022-07-11,BY,9999538518,7.05465kE0KaI2BaGckd,4945,0,0
...,...,...,...,...,...,...,...
45538,2022-08-20,US,1000048325,4AibH91K4Ifk81C7_c.1,107,0,0
9,2022-04-17,CC,0,.3k8Egg2ahGH1DF08Dck,2264,1,0
84,2022-06-06,CA,0,h21iChHb974dDb9F3E.g,3752,1,0
16636,2022-11-18,GR,(cid),B3a798eD1C0D-EkDH5E4,1392,0,0


Видим еще значение,которое не подходит по типу -'𝐶𝐴𝑀𝑃𝐴𝐼𝐺𝑁𝐼𝐷'.

Работаем с колонкой country.
country-двухбуквенный код страны в формате ISO 3166.
Тут у нас должны быть значения двухсимвольные и которые содержатся в общепринятом списке стран в формате ISO 3166.

Этот список можно скачать в интернете в формате csv файла, загрузить и сравнить со значениями столбца. Могут быть обозначения стран, которые не сущетсвуют или вообще написаны не в нужном формате.

In [256]:
country_df=pd.read_csv('country.csv',delimiter=';',header=None)
country_df.head(2)

,0,1
0,Andorra,AD
1,United Arab Emirates,AE


In [260]:
countries=[]
#no_countries=[]
countries=country_df[1].tolist()
#for i in countries:
no_country_df=~test_df['country'].isin(countries)
test_df[no_country_df]

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
10,2022-04-13,UK,9466735046,A230bchBEh03-1dek876,4882,0,0
38,2022-09-02,59,3740410419,Dc_Ei_cC8cHcK2A5hB8F,1846,0,0
49,2022-04-05,UK,3054555256,k.5B5b0di89kF0CG09Hk,1926,0,0
64,2023-01-24,UK,2907487746,aCbI-HdI6g2E44aD8844,1400,0,0
82,2023-02-07,UK,9183178244,DKeDa1cB._Ai1_59-IAf,141,0,0
...,...,...,...,...,...,...,...
99977,2023-03-02,UK,3722321704,KI0_k1154B0Hgad_0.8E,3918,0,0
99985,2022-12-14,ZZ,5321546943,Fg-Dgg.ed459Bh713h1E,204,0,0
99991,2023-01-31,UK,5939447062,5deB-1-_IFBaB-Ab6Dhh,840,0,0
99992,2022-06-26,UK,5562661870,3HDKkBH_d9I6GA8ab24.,1990,0,0


In [261]:
test_df[no_country_df].groupby(['country']).mean()

,installs,campaign_id_nan_flag,country_nan_flag
country,,,
0,2526.657286,0.0,1.0
59,2531.959325,0.0,0.0
EU,2466.372605,0.0,0.0
UK,2474.512654,0.0,0.0
ZZ,2449.523671,0.0,0.0


Значение 0 не мешает. Так как указали,что это строки, где не было значений в колонке country.
Значение 59 - не подходит к типу данных и не имеет смысла.
Значения EU,UK,ZZ - нет таких стран в формате ISO 3166. Имелось что-то другое...

In [259]:
test_df.head()

,install_date,country,campaign_id,campaign_name,installs,campaign_id_nan_flag,country_nan_flag
0,2022-10-15,CH,6488890999,I3cE61G6kA1ihaBcdDAA,256,0,0
1,2022-08-30,CX,6896175429,eH0Bk-G..d7GCgA5kff2,1022,0,0
2,2022-05-23,BY,2922650240,DF6_h_E.bf0_._cA2C7b,3837,0,0
3,2023-01-23,CX,8282076813,GBFgbGb4ee2.G.G2aAFK,272,0,0
4,2022-11-27,BR,7617371497,H2kb10__.fGHc88f77fb,3984,0,0
